In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from scipy import stats
import glob
import os
import os.path
from operator import add
from scipy.optimize import curve_fit
from PIL import Image
import time
import sys
#import curses
#import Module_Test as m
from scipy.stats import norm
import matplotlib.mlab as mlab
from math import e, pi, isinf
#stdscr = curses.initscr()
import math
import statistics 
from matplotlib.offsetbox import AnchoredText


def best_fit(X, Y):

    xbar = sum(X)/len(X)
    ybar = sum(Y)/len(Y)
    n = len(X) # or len(Y)

    numer = sum([xi*yi for xi,yi in zip(X, Y)]) - n * xbar * ybar
    denum = sum([xi**2 for xi in X]) - n * xbar**2

    b = numer / denum
    a = ybar - b * xbar

    #print('best fit line:\ny = {:.7f} + {:.7f}x'.format(a, b))

    return a, b

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def attn_factor(value):
    c = (10**(value/20))
    return c

def gauss(x,*p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2*sigma**2))

In [2]:
print("")
FileLocation = input("Input File Location: ")
os.chdir(FileLocation)
print("")
print("")
print("For this part, type the number of each channel you are using.\nExample: if you are using all channels type 1234, or if you are only using 1, 2 and 4, type 124.")
print("")
Squares = input("Insert the channels you are using: ")
print("")
print("This is where a noise floor will be set.\nExample: if you want a floor at 75 mV, type .075 :")
Noise = float(input("Where would you like to set the noise floor?:"))
print("")


Input File Location: /media/ilker/Ilker/PMT_Characterization/with_light_FW46_FS9_1200/tube_withteflon/10k


For this part, type the number of each channel you are using.
Example: if you are using all channels type 1234, or if you are only using 1, 2 and 4, type 124.

Insert the channels you are using: 4

This is where a noise floor will be set.
Example: if you want a floor at 75 mV, type .075 :
Where would you like to set the noise floor?:0



In [ ]:
Nc1 = int(len(glob.glob('C1*'))) #Finds the number of trace files in directory
Nc2 = int(len(glob.glob('C2*')))
Nc3 = int(len(glob.glob('C3*')))
Nc4 = int(len(glob.glob('C4*')))
if(Nc1 != 0):
    Progressor = (1 / (len(Squares) * Nc1)) * 10
    Length = Nc1
elif(Nc2 != 0):
    Progressor = (1 / (len(Squares) * Nc2)) * 10
    Length = Nc2
elif(Nc3 != 0):
    Progressor = (1 / (len(Squares) * Nc3)) * 10
    Length = Nc3
elif(Nc4 != 0):
    Progressor = (1 / (len(Squares) * Nc4)) * 10
    Length = Nc4

In [ ]:
File1 = []
File2 = []
File3 = []
File4 = []
if(Squares.find("1")!= -1):
    for file in glob.glob("C1*"):
        File1.append(file)
        
if(Squares.find("2")!= -1):
    for file in glob.glob("C2*"):
        File2.append(file)
        
if(Squares.find("3")!= -1):
    for file in glob.glob("C3*"):
        File3.append(file)
        
if(Squares.find("4")!= -1):
    for file in glob.glob("C4*"):
        File4.append(file)

In [ ]:
Voltage1 = []
Voltage2 = []
Voltage3 = []
Voltage4 = []
Time1 = []
Time2 = []
Time3 = []
Time4 = []
VoltageA = []
VoltageB = []
VoltageC = []
VoltageD = []
TimeA = []
TimeB = []
TimeC = []
TimeD = []
MinV1 = []
MinV2 = []
MinV3 = []
MinV4 = []
Count2 = 0
CountV2 = 0


Start = time.time()
InputFile = []
if (Squares.find("1") != -1):
    inFile = open(File1[0]) #Imports data set size numbers
    #dT = inFile
    for line in inFile:
        if line.startswith("Segments,"):
            InputFile = line
elif (Squares.find("2") != -1):
    inFile = open(File2[0])
    for line in inFile:
        if line.startswith("Segments,"):
            InputFile = line
elif (Squares.find("3") != -1):
    inFile = open(File3[0])
    for line in inFile:
        if line.startswith("Segments,"):
            InputFile = line
elif (Squares.find("4") != -1):
    inFile = open(File4[0])
    for line in inFile:
        if line.startswith("Segments,"):
            InputFile = line

if (Squares.find("1") != -1):           
    Times1 = pd.DataFrame(np.array(pd.read_csv(File1[0] ,header=(int(InputFile.split(",")[1]) + 3), index_col=False, low_memory=False)))
    dT = Times1[0][1] - Times1[0][0]
if (Squares.find("2") != -1):           
    Times2 = pd.DataFrame(np.array(pd.read_csv(File2[0] ,header=(int(InputFile.split(",")[1]) + 3), index_col=False, low_memory=False)))
    dT = Times2[0][1] - Times2[0][0]
if (Squares.find("3") != -1):           
    Times3 = pd.DataFrame(np.array(pd.read_csv(File3[0] ,header=(int(InputFile.split(",")[1]) + 3), index_col=False, low_memory=False)))
    dT = Times3[0][1] - Times3[0][0]
if (Squares.find("4") != -1):           
    Times4 = pd.DataFrame(np.array(pd.read_csv(File4[0] ,header=(int(InputFile.split(",")[1]) + 3), index_col=False, low_memory=False)))
    dT = Times4[0][1] - Times4[0][0]
    
    
Reshaper = int(InputFile.split(",")[3]) #Pulls the length of a sample size from an input file
Reshaper = int(Reshaper)
inFile.close()

for k in range(0, Length):
    print("", end="\r", flush=True)
    print(k,  end="", flush=True)
    Count2 = 0
    Total2 = 0
    Count4 = 0
    Total4 = 0
    CountV2 = 0
    if (Squares.find("1") != -1):
        Voltage1 = []
        Time1 = []
        Sig1 = pd.read_csv(File1[k] ,header=(int(InputFile.split(",")[1]) + 3), index_col=False, low_memory=False)
        Sig1 = np.array(Sig1)
        Voltage1 = np.append(Voltage1, Sig1[:,1], axis=0)
        Voltage1 = np.array(Voltage1).reshape(-1, Reshaper)
        Voltage1 = pd.DataFrame(Voltage1)
        Voltage1 = Voltage1.transpose()
        Time1 = np.append(Time1, Sig1[:,0], axis = 0)
        Time1=np.array(Time1).reshape(-1, Reshaper)
        Time1 = pd.DataFrame(Time1)
        Time1 = Time1.transpose()
        #print("Worked1!")
    if (Squares.find("2") != -1):
        Voltage2 = []
        Time2 = []
        Sig2 = pd.read_csv(File2[k] ,header=(int(InputFile.split(",")[1]) + 3), index_col=False, low_memory=False)
        Sig2 = np.array(Sig2)
        Voltage2 = np.append(Voltage2, Sig2[:,1], axis=0)
        Voltage2 = np.array(Voltage2).reshape(-1, Reshaper)
        Voltage2 = pd.DataFrame(Voltage2)
        Voltage2 = Voltage2.transpose()
        Time2 = np.append(Time2, Sig2[:,0], axis = 0)
        Time2 = np.array(Time2).reshape(-1, Reshaper)
        Time2 = pd.DataFrame(Time2)
        Time2 = Time2.transpose()
        
        #print("Worked2!")
    if (Squares.find("3") != -1):
        Voltage3 = []
        Time3 = []
        Sig3 = pd.read_csv(File3[k],header=(int(InputFile.split(",")[1]) + 3), index_col=False, low_memory=False)
        Sig3 = np.array(Sig3)
        Voltage3 = np.append(Voltage3, Sig3[:,1], axis=0)
        Voltage3 = np.array(Voltage3).reshape(-1, Reshaper)
        Voltage3 = pd.DataFrame(Voltage3)
        Voltage3 = Voltage3.transpose()
        Time3 = np.append(Time3, Sig3[:,0], axis = 0)
        Time3 = np.array(Time3).reshape(-1, Reshaper)
        Time3 = pd.DataFrame(Time3)
        Time3 = Time3.transpose()
        #print("Worked3!")
    if (Squares.find("4") != -1):
        Voltage4 = []
        Time4 = []
        Sig4 = pd.read_csv(File4[k] ,header=(int(InputFile.split(",")[1]) + 3), index_col=False, low_memory=False)
        Sig4 = np.array(Sig4)
        Voltage4 = np.append(Voltage4, Sig4[:,1], axis=0)
        Voltage4 = np.array(Voltage4).reshape(-1, Reshaper)
        Voltage4 = pd.DataFrame(Voltage4)
        Voltage4 = Voltage4.transpose()
        Time4 = np.append(Time4, Sig4[:,0], axis = 0)
        Time4 = np.array(Time4).reshape(-1, Reshaper)
        Time4 = pd.DataFrame(Time4)
        Time4 = Time4.transpose()
        #print("Worked4!")
    if (Squares.find("1") != -1):
        for i in range(0, len(Voltage1.columns)):
            if(abs(min(Voltage1[i])) > Noise):
                if(np.where(Voltage1[i] == min(Voltage1[i]))[0][0] != 0):
                    VoltageA.append(Voltage1[i])
                    TimeA.append(Time1[i])
    if (Squares.find("2") != -1):
        for i in range(0, len(Voltage2.columns)):
            if(abs(min(Voltage2[i])) > Noise):
                CountV2 = CountV2 + 1
                if(np.where(Voltage2[i] == min(Voltage2[i]))[0][0] != 0):
                    VoltageB.append(Voltage2[i])
                    TimeB.append(Time2[i])
                    Count2 = Count2 + 1
    if (Squares.find("3") != -1):
        for i in range(0, len(Voltage3.columns)):
            if(abs(min(Voltage3[i])) > Noise):
                if(np.where(Voltage3[i] == min(Voltage3[i]))[0][0] != 0):
                    VoltageC.append(Voltage3[i])
                    TimeC.append(Time3[i])
    if (Squares.find("4") != -1):
        for i in range(0, len(Voltage4.columns)):
            if(abs(min(Voltage4[i])) > Noise):
                if(np.where(Voltage4[i] == min(Voltage4[i]))[0][0] != 0):
                    VoltageD.append(Voltage4[i])
                    TimeD.append(Time4[i])
                    Count4 = Count4 + 1
    #print(len(Voltage2.columns) - Count2,Count2,  CountV2)
    #print(len(Voltage4.columns) - Count4)

In [ ]:
if (Squares.find("1") != -1): 
    voltageA=pd.DataFrame(np.array(VoltageA).reshape(-1, Reshaper)).transpose()
    timeA=pd.DataFrame(np.array(TimeA).reshape(-1, Reshaper)).transpose()
    

if (Squares.find("2") != -1): 
    voltageB=pd.DataFrame(np.array(VoltageB).reshape(-1, Reshaper)).transpose()
    timeB=pd.DataFrame(np.array(TimeB).reshape(-1, Reshaper)).transpose()

if (Squares.find("3") != -1): 
    voltageC=pd.DataFrame(np.array(VoltageC).reshape(-1, Reshaper)).transpose()
    timeC=pd.DataFrame(np.array(TimeC).reshape(-1, Reshaper)).transpose()
    
if (Squares.find("4") != -1): 
    voltageD=pd.DataFrame(np.array(VoltageD).reshape(-1, Reshaper)).transpose()
    timeD=pd.DataFrame(np.array(TimeD).reshape(-1, Reshaper)).transpose()

In [ ]:
VoltageA = []
VoltageB = []
VoltageC = []
VoltageD = []
TimeA = []
TimeB = []
TimeC = []
TimeD = []

In [ ]:

if (Squares.find("1") != -1): 
    Record_length = len(voltageA.index)
    sweeps = len(voltageA.columns)
    new_time_array1=[]        # The array that the new times will go into
    for i in range (0,sweeps):
        #a=('%d'%i)
        V_Min=min(voltageA[i])                          # Finds the minimum voltage of voltageT
        VML=np.where(voltageA[i]==V_Min)[0]             # Finds the location of the minimum voltage
        Loc_Int=VML[0]                                  # Turns the location value into an integer
        V_Column_Array=voltageA[i]                      # Sets the current voltage column
        T_Column_Array=timeA[i]                         # Sets the current time column
        Half_V_Min=.42*min(V_Column_Array)               # Finds percentage of the minimum voltage value
        first_loc=np.where(V_Column_Array == find_nearest(V_Column_Array[0:Loc_Int],Half_V_Min))[0][0]  # Locates the point closest to half of the minimum voltage value
        if(first_loc != 0):
            first_point_V=V_Column_Array[first_loc]    # Sets the value of the voltage for the first point 
            first_point_T=T_Column_Array[first_loc]    # Sets the value of the time for the first point
            if (first_point_V<Half_V_Min):                  # Determines where to get the second points based on first points relation to the half minimum voltage
                second_point_V=V_Column_Array[first_loc-1] # Sets the value of the voltage for the second point if the first point voltage is lower the half minimum voltage
                second_point_T=T_Column_Array[first_loc-1] # Sets the value of the time for the second point if the first point voltage is lower the half minimum voltage
            else:
                second_point_V=V_Column_Array[first_loc+1] # Sets the value of the voltage for the second point if the first point voltage is higher the half minimum voltage
                second_point_T=T_Column_Array[first_loc+1] # Sets the value of the time for the second point if the first point voltage is higher the half minimum voltage
            slope=(second_point_V-first_point_V)/(second_point_T-first_point_T) # Finds the slope of line between the first and second points
            new_time=(1/slope)*(Half_V_Min-(first_point_V-(slope*first_point_T))) # Calculates the time when the voltage was at half of the minimum voltage value
            new_time_array1.append(new_time) # Appends the new_time_array with the new calculated times

In [ ]:
voltageA = []
timeA = []

In [ ]:

if (Squares.find("2") != -1): 
    Record_length = len(voltageB.index)
    sweeps = len(voltageB.columns)
    new_time_array2=[]        # The array that the new times will go into
    for i in range (0,sweeps):
        #a=('%d'%i)
        V_Min=min(voltageB[i])                          # Finds the minimum voltage of voltageT
        VML=np.where(voltageB[i]==V_Min)[0]             # Finds the location of the minimum voltage
        Loc_Int=VML[0]                                  # Turns the location value into an integer
        V_Column_Array=voltageB[i]                      # Sets the current voltage column
        T_Column_Array=timeB[i]                         # Sets the current time column
        Half_V_Min=.42*min(V_Column_Array)               # Finds percentage of the minimum voltage value
        first_loc=np.where(V_Column_Array == find_nearest(V_Column_Array[0:Loc_Int],Half_V_Min))[0][0]  # Locates the point closest to half of the minimum voltage value
        if(first_loc != 0):
            first_point_V=V_Column_Array[first_loc]    # Sets the value of the voltage for the first point 
            first_point_T=T_Column_Array[first_loc]    # Sets the value of the time for the first point
            if (first_point_V<Half_V_Min):                  # Determines where to get the second points based on first points relation to the half minimum voltage
                second_point_V=V_Column_Array[first_loc-1] # Sets the value of the voltage for the second point if the first point voltage is lower the half minimum voltage
                second_point_T=T_Column_Array[first_loc-1] # Sets the value of the time for the second point if the first point voltage is lower the half minimum voltage
            else:
                second_point_V=V_Column_Array[first_loc+1] # Sets the value of the voltage for the second point if the first point voltage is higher the half minimum voltage
                second_point_T=T_Column_Array[first_loc+1] # Sets the value of the time for the second point if the first point voltage is higher the half minimum voltage
            slope=(second_point_V-first_point_V)/(second_point_T-first_point_T) # Finds the slope of line between the first and second points
            new_time=(1/slope)*(Half_V_Min-(first_point_V-(slope*first_point_T))) # Calculates the time when the voltage was at half of the minimum voltage value
            new_time_array2.append(new_time) # Appends the new_time_array with the new calculated times

In [ ]:
voltageB = []
timeB = []

In [ ]:

if (Squares.find("3") != -1):
    Record_length = len(voltageC.index)
    sweeps = len(voltageC.columns)
    new_time_array3=[]        # The array that the new times will go into
    for i in range (0,sweeps):
        #a=('%d'%i)
        V_Min=min(voltageC[i])                          # Finds the minimum voltage of voltageT
        VML=np.where(voltageC[i]==V_Min)[0]             # Finds the location of the minimum voltage
        Loc_Int=VML[0]                                  # Turns the location value into an integer
        V_Column_Array=voltageC[i]                      # Sets the current voltage column
        T_Column_Array=timeC[i]                         # Sets the current time column
        Half_V_Min=.42*min(V_Column_Array)               # Finds percentage of the minimum voltage value
        first_loc=np.where(V_Column_Array == find_nearest(V_Column_Array[0:Loc_Int],Half_V_Min))[0][0]  # Locates the point closest to half of the minimum voltage value
        if(first_loc != 0):
            first_point_V=V_Column_Array[first_loc]    # Sets the value of the voltage for the first point 
            first_point_T=T_Column_Array[first_loc]    # Sets the value of the time for the first point
            if (first_point_V<Half_V_Min):                  # Determines where to get the second points based on first points relation to the half minimum voltage
                second_point_V=V_Column_Array[first_loc-1] # Sets the value of the voltage for the second point if the first point voltage is lower the half minimum voltage
                second_point_T=T_Column_Array[first_loc-1] # Sets the value of the time for the second point if the first point voltage is lower the half minimum voltage
            else:
                second_point_V=V_Column_Array[first_loc+1] # Sets the value of the voltage for the second point if the first point voltage is higher the half minimum voltage
                second_point_T=T_Column_Array[first_loc+1] # Sets the value of the time for the second point if the first point voltage is higher the half minimum voltage
            slope=(second_point_V-first_point_V)/(second_point_T-first_point_T) # Finds the slope of line between the first and second points
            new_time=(1/slope)*(Half_V_Min-(first_point_V-(slope*first_point_T))) # Calculates the time when the voltage was at half of the minimum voltage value
            new_time_array3.append(new_time) # Appends the new_time_array with the new calculated times

In [ ]:
voltageC = []
timeC = []

In [ ]:
if (Squares.find("4") != -1):
    Record_length = len(voltageD.index)
    sweeps = len(voltageD.columns)
    new_time_array4=[]        # The array that the new times will go into
    for i in range (0,sweeps):
        #a=('%d'%i)
        V_Min=min(voltageD[i])                          # Finds the minimum voltage of voltageT
        VML=np.where(voltageD[i]==V_Min)[0]             # Finds the location of the minimum voltage
        Loc_Int=VML[0]                                  # Turns the location value into an integer
        V_Column_Array=voltageD[i]                      # Sets the current voltage column
        T_Column_Array=timeD[i]                         # Sets the current time column
        Half_V_Min=.42*min(V_Column_Array)               # Finds percentage of the minimum voltage value
        first_loc=np.where(V_Column_Array == find_nearest(V_Column_Array[0:Loc_Int],Half_V_Min))[0][0]  # Locates the point closest to half of the minimum voltage value
        if(first_loc != 0):
            first_point_V=V_Column_Array[first_loc]    # Sets the value of the voltage for the first point 
            first_point_T=T_Column_Array[first_loc]    # Sets the value of the time for the first point
            if (first_point_V<Half_V_Min):                  # Determines where to get the second points based on first points relation to the half minimum voltage
                second_point_V=V_Column_Array[first_loc-1] # Sets the value of the voltage for the second point if the first point voltage is lower the half minimum voltage
                second_point_T=T_Column_Array[first_loc-1] # Sets the value of the time for the second point if the first point voltage is lower the half minimum voltage
            else:
                second_point_V=V_Column_Array[first_loc+1] # Sets the value of the voltage for the second point if the first point voltage is higher the half minimum voltage
                second_point_T=T_Column_Array[first_loc+1] # Sets the value of the time for the second point if the first point voltage is higher the half minimum voltage
            slope=(second_point_V-first_point_V)/(second_point_T-first_point_T) # Finds the slope of line between the first and second points
            new_time=(1/slope)*(Half_V_Min-(first_point_V-(slope*first_point_T))) # Calculates the time when the voltage was at half of the minimum voltage value
            new_time_array4.append(new_time) # Appends the new_time_array with the new calculated times

In [ ]:
voltageD = []
timeD = []

In [ ]:
if(Squares.find("1") != -1):
    T1 = new_time_array1

if(Squares.find("2") != -1):
    T2 = new_time_array2
    
if(Squares.find("3") != -1):
    T3 = new_time_array3
    
if(Squares.find("4") != -1):
    T4 = new_time_array4

In [ ]:
T1T = []
T2T = []
T3T = []
T4T = []


if(Squares.find("1") != -1):
    for i in range(0,len(T1)):
        if(isinf(T1[i])):
            X = 0
        else:
            T1T.append(T1[i])
    T1 = T1T

if(Squares.find("2") != -1):
    for i in range(0,len(T2)):
        if(isinf(T2[i])):
            X = 0
        else:
            T2T.append(T2[i])
    T2 = T2T

if(Squares.find("3") != -1):
    for i in range(0,len(T3)):
        if(isinf(T3[i])):
            X = 0
        else:
            T3T.append(T3[i])
    T3 = T3T
    
if(Squares.find("4") != -1):
    for i in range(0,len(T4)):
        if(isinf(T4[i])):
            X = 0
        else:
            T4T.append(T4[i])
    T4 = T4T


In [ ]:
if(Squares.find("1") != -1):
    
    #histo, bin_edges = np.histogram(T2, 500)
    #Mass = 0
    #for i in range (0,len(histo)):
        #Mass = Mass + histo[i]*i
    #middle = int(Mass/sum(histo))
    #width = middle+int(middle * .5)
    #width = [width]
    #width2 = middle-int(middle * .5)
    #width2 = [width2]
    fig, ax = plt.subplots(figsize=[12,8])
    histo, bin_edges = np.histogram(T1, 1000)
    middle = np.where(histo == max(histo))[0]
    width = np.where(histo == find_nearest(histo[middle[0]:999] , .5 * max(histo)))[0]
    width2 = np.where(histo == find_nearest(histo[0:middle[0]] , .5 * max(histo)))[0]
    max_histo = max(histo)
    
    n, bins, patches = ax.hist(T1, 1000, fill=False)
    #Line below is used to obtain bin edge locations
    #histo, bin_edges = np.histogram(T2, bins)
    #Line below uses knowledge of bin edge locations to find bin centers
    bin_centers = (bin_edges[:-1]+bin_edges[1:])/2
    
    #coeff, var_matrix = curve_fit(gauss, bin_centers[width2:width], histo[width2:width], p0=[middle,np.mean(T2),2e-4])

    #defines the range of x-axis over which to place the fit curve
    #Use the mean calculated above to make initial guess at sensible values
    #adjust as necessary
    #coeff, var_matrix = curve_fit(gauss,bin_centers, histo, p0 = [middle, np.mean(T2), .5e-8])
    #coeff, var_matrix = curve_fit(gauss, bin_centers[width2[0]-1:width[0]], histo[width2[0]-1:width[0]], p0=[250,np.mean(T1), 2 * (bin_centers[1] - bin_centers[0]) / 2.355])
    vars = np.linspace(bin_centers[width[0]],bin_centers[width2[0]-1])
    #hist_fit = gauss(vars,*coeff)
    #ax.plot(vars,hist_fit, 'r', linewidth=2)
    plt.title("Ch1_Time_%0.3fV_Cut" %(Noise), fontsize = "18")
    plt.xlabel('Time (ns)', fontsize ='18')
    plt.ylabel('Events/bin', fontsize ='18')
    #text_box = AnchoredText("Number of Events: %d \nFitted Sigma: %0.3e" %(sum(histo), abs(coeff[2])), frameon=True, loc=1, pad=0.5)
    #plt.setp(text_box.patch, facecolor='white', alpha=0.5)
    #ax.add_artist(text_box)
    #plt.text(np.mean(T1) + (abs(coeff[2])*2), .9 *max(histo), 'Fitted sigma = {0:1.3e}'.format(abs(coeff[2])),bbox=dict(facecolor='white', alpha=1))
    #plt.xlim(np.mean(T3)-(abs(coeff[2])*6),np.mean(T3)+(abs(coeff[2])*6))
    plt.savefig("Ch1_Time_%0.3fV_Cut.png" %(Noise))
    plt.show()
    #CFD time@level distro

    #print('Fitted mean = ', coeff[1])
    #print('Fitted sigma = ', coeff[2])
    print('Bin width = ', bin_edges[2] - bin_edges[1])
    print('N = ', len(T1))
    histo = pd.DataFrame(histo)
    bin_centers = pd.DataFrame(bin_centers)
    histo.to_csv("Ch1_Time_%0.3fV_Cut_Histo" %(Noise), index = False)
    bin_centers.to_csv("Ch1_Time_%0.3fV_Cut_Bin_Centers" %(Noise), index = False)


In [ ]:
if(Squares.find("2") != -1):
    
    #histo, bin_edges = np.histogram(T2, 500)
    #Mass = 0
    #for i in range (0,len(histo)):
        #Mass = Mass + histo[i]*i
    #middle = int(Mass/sum(histo))
    #width = middle+int(middle * .5)
    #width = [width]
    #width2 = middle-int(middle * .5)
    #width2 = [width2]
    fig, ax = plt.subplots(figsize=[12,8])
    histo, bin_edges = np.histogram(T2, 500)
    middle = np.where(histo == max(histo))[0]
    width = np.where(histo == find_nearest(histo[middle[0]:999] , .5 * max(histo)))[0]
    width2 = np.where(histo == find_nearest(histo[0:middle[0]] , .5 * max(histo)))[0]
    max_histo = max(histo)
    
    n, bins, patches = ax.hist(T2, 500, fill=False)
    #Line below is used to obtain bin edge locations
    #histo, bin_edges = np.histogram(T2, bins)
    #Line below uses knowledge of bin edge locations to find bin centers
    bin_centers = (bin_edges[:-1]+bin_edges[1:])/2
    
    #coeff, var_matrix = curve_fit(gauss, bin_centers[width2:width], histo[width2:width], p0=[middle,np.mean(T2),2e-4])

    #defines the range of x-axis over which to place the fit curve
    #Use the mean calculated above to make initial guess at sensible values
    #adjust as necessary
    #coeff, var_matrix = curve_fit(gauss,bin_centers, histo, p0 = [middle, np.mean(T2), .5e-8])
    #coeff, var_matrix = curve_fit(gauss, bin_centers[width2[0]-1:width[0]], histo[width2[0]-1:width[0]], p0=[250,np.mean(T2), 2 * (bin_centers[1] - bin_centers[0]) / 2.355])
    #vars = np.linspace(bin_centers[width[0]],bin_centers[width2[0]-1])
    #hist_fit = gauss(vars,*coeff)
    #ax.plot(vars,hist_fit, 'r', linewidth=2)
    plt.title("Ch2_Time_%0.3fV_Cut_Time_Cut" %(Noise), fontsize = "18")
    plt.xlabel('Time (ns)', fontsize ='18')
    plt.ylabel('Events/bin', fontsize ='18')
    #text_box = AnchoredText("Number of Events: %d \nFitted Sigma: %0.3e" %(sum(histo), abs(coeff[2])), frameon=True, loc=1, pad=0.5)
    #plt.setp(text_box.patch, facecolor='white', alpha=0.5)
    #ax.add_artist(text_box)
    #plt.text(np.mean(T2) + (abs(coeff[2])*2), .9 *max(histo), 'Fitted sigma = {0:1.3e}'.format(abs(coeff[2])),bbox=dict(facecolor='white', alpha=1))
    #plt.xlim(np.mean(T3)-(abs(coeff[2])*6),np.mean(T3)+(abs(coeff[2])*6))
    plt.savefig("Ch2_Time_%0.3fV_Cut_Time_Cut.png" %(Noise))
    plt.show()
    #CFD time@level distro

    #print('Fitted mean = ', coeff[1])
    #print('Fitted sigma = ', coeff[2])
    #print('Bin width = ', bin_edges[2] - bin_edges[1])
    #print('N = ', len(T2))
    histo = pd.DataFrame(histo)
    bin_centers = pd.DataFrame(bin_centers)
    histo.to_csv("Ch2_Time_%0.3fV_Cut_Time_Cut_Histo_3" %(Noise), index = False)
    bin_centers.to_csv("Ch2_Time_%0.3fV_Cut_TIme_Cut_Bin_Centers_3" %(Noise), index = False)


In [ ]:
if(Squares.find("3") != -1):
    
    #histo, bin_edges = np.histogram(T2, 500)
    #Mass = 0
    #for i in range (0,len(histo)):
        #Mass = Mass + histo[i]*i
    #middle = int(Mass/sum(histo))
    #width = middle+int(middle * .5)
    #width = [width]
    #width2 = middle-int(middle * .5)
    #width2 = [width2]
    fig, ax = plt.subplots(figsize=[12,8])
    histo, bin_edges = np.histogram(T3, 1000)
    middle = np.where(histo == max(histo))[0]
    width = np.where(histo == find_nearest(histo[middle[0]:999] , .5 * max(histo)))[0]
    width2 = np.where(histo == find_nearest(histo[0:middle[0]] , .5 * max(histo)))[0]
    max_histo = max(histo)
    
    n, bins, patches = ax.hist(T3, 1000, fill=False)
    #Line below is used to obtain bin edge locations
    #histo, bin_edges = np.histogram(T2, bins)
    #Line below uses knowledge of bin edge locations to find bin centers
    bin_centers = (bin_edges[:-1]+bin_edges[1:])/2
    
    #coeff, var_matrix = curve_fit(gauss, bin_centers[width2:width], histo[width2:width], p0=[middle,np.mean(T2),2e-4])

    #defines the range of x-axis over which to place the fit curve
    #Use the mean calculated above to make initial guess at sensible values
    #adjust as necessary
    #coeff, var_matrix = curve_fit(gauss,bin_centers, histo, p0 = [middle, np.mean(T2), .5e-8])
    #coeff, var_matrix = curve_fit(gauss, bin_centers[width2[0]-1:width[0]], histo[width2[0]-1:width[0]], p0=[250,np.mean(T3), 2 * (bin_centers[1] - bin_centers[0]) / 2.355])
    #vars = np.linspace(bin_centers[width[0]],bin_centers[width2[0]-1])
    #hist_fit = gauss(vars,*coeff)
    #ax.plot(vars,hist_fit, 'r', linewidth=2)
    plt.title("Ch3_Time_%0.3fV_Cut" %(Noise), fontsize = "18")
    plt.xlabel('Time (ns)', fontsize ='18')
    plt.ylabel('Events/bin', fontsize ='18')
    #text_box = AnchoredText("Number of Events: %d \nFitted Sigma: %0.3e" %(sum(histo), abs(coeff[2])), frameon=True, loc=1, pad=0.5)
    #plt.setp(text_box.patch, facecolor='white', alpha=0.5)
    #ax.add_artist(text_box)
    #plt.text(np.mean(T3) + (abs(coeff[2])*2), .9 *max(histo), 'Fitted sigma = {0:1.3e}'.format(abs(coeff[2])),bbox=dict(facecolor='white', alpha=1))
    #plt.xlim(np.mean(T3)-(abs(coeff[2])*6),np.mean(T3)+(abs(coeff[2])*6))
    plt.savefig("Ch3_Time_%0.3fV_Cut.png" %(Noise))
    plt.show()
    #CFD time@level distro

    #print('Fitted mean = ', coeff[1])
    #print('Fitted sigma = ', coeff[2])
    print('Bin width = ', bin_edges[2] - bin_edges[1])
    print('N = ', len(T3))
    histo = pd.DataFrame(histo)
    bin_centers = pd.DataFrame(bin_centers)
    histo.to_csv("Ch3_Time_%0.3fV_Cut_Histo" %(Noise), index = False)
    bin_centers.to_csv("Ch3_Time_%0.3fV_Cut_Bin_Centers" %(Noise), index = False)


In [ ]:
if(Squares.find("4") != -1):
    
    #histo, bin_edges = np.histogram(T2, 500)
    #Mass = 0
    #for i in range (0,len(histo)):
        #Mass = Mass + histo[i]*i
    #middle = int(Mass/sum(histo))
    #width = middle+int(middle * .5)
    #width = [width]
    #width2 = middle-int(middle * .5)
    #width2 = [width2]
    fig, ax = plt.subplots(figsize=[12,8])
    histo, bin_edges = np.histogram(T4, 1000)
    middle = np.where(histo == max(histo))[0]
    width = [middle[0] + np.where(histo[middle[0]:999] == find_nearest(histo[middle[0]:999] , .5 * max(histo)))[0][0]]
    width2 = np.where(histo[0:middle[0]] == find_nearest(histo[0:middle[0]] , .5 * max(histo)))[0]
    max_histo = max(histo)
    
    n, bins, patches = ax.hist(T4, 1000, fill=False)
    #Line below is used to obtain bin edge locations
    #histo, bin_edges = np.histogram(T2, bins)
    #Line below uses knowledge of bin edge locations to find bin centers
    bin_centers = (bin_edges[:-1]+bin_edges[1:])/2
    
    #coeff, var_matrix = curve_fit(gauss, bin_centers[width2:width], histo[width2:width], p0=[middle,np.mean(T2),2e-4])

    #defines the range of x-axis over which to place the fit curve
    #Use the mean calculated above to make initial guess at sensible values
    #adjust as necessary
    #coeff, var_matrix = curve_fit(gauss,bin_centers, histo, p0 = [middle, np.mean(T2), .5e-8])
    #coeff, var_matrix = curve_fit(gauss, bin_centers[width2[0]:width[0]], histo[width2[0]:width[0]], p0=[1000,np.mean(T4), 2 * (bin_centers[1] - bin_centers[0])])
    vars = np.linspace(bin_centers[width[0]],bin_centers[width2[0]])
    #hist_fit = gauss(vars,*coeff)
    #ax.plot(vars,hist_fit, 'r', linewidth=2)
    plt.title("Ch4_Time_%0.3fV_Cut" %(Noise), fontsize = "18")
    plt.xlabel('Time (ns)', fontsize ='18')
    plt.ylabel('Events/bin', fontsize ='18')
    #plt.xlim(.4*10**-8, .75*10**-8)
    #text_box = AnchoredText("Number of Events: %d \nFitted Sigma: %0.3e" %(sum(histo), abs(coeff[2])), frameon=True, loc=1, pad=0.5)
    #plt.setp(text_box.patch, facecolor='white', alpha=0.5)
    #ax.add_artist(text_box)
    #plt.text(np.mean(T4) + (abs(coeff[2])*2), .9 *max(histo), 'Fitted sigma = {0:1.3e}'.format(abs(coeff[2])),bbox=dict(facecolor='white', alpha=1))
    #plt.xlim(np.mean(T3)-(abs(coeff[2])*6),np.mean(T3)+(abs(coeff[2])*6))
    plt.savefig("Ch4_Time_%0.3fV_Cut.png" %(Noise))
    plt.show()
    #CFD time@level distro

    #print('Fitted mean = ', coeff[1])
    #print('Fitted sigma = ', coeff[2])
    #print('Bin width = ', bin_edges[2] - bin_edges[1])
    #print('N = ', len(T4))
    histo = pd.DataFrame(histo)
    bin_centers = pd.DataFrame(bin_centers)
    histo.to_csv("Ch4_Time_%0.3fV_Cut_Histo" %(Noise), index = False)
    bin_centers.to_csv("Ch4_Time_%0.3fV_Cut_Bin_Centers" %(Noise), index = False)


In [ ]:
for i in range(0,100):
    plt.plot(Time2[i], Voltage2[i])
    plt.show()

In [ ]:
Time1

In [ ]:
Volt